In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from neural_network import NeuralNetworkModel, ChessDataset, ChessEvaluationNeuralNetwork

In [20]:
def eval_to_output(eval: str):
    if eval.startswith('#+'):
        return 10000.0
    elif eval.startswith('#-'):
        return -10000.0
    else:
        return float(eval)

In [21]:
df = pd.read_csv("../../data/chess_data.csv")

X = df['FEN'].apply(NeuralNetworkModel.fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(eval_to_output).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [22]:
model = ChessEvaluationNeuralNetwork(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 934807.0217
Epoch 2, Loss: 874156.7785
Epoch 3, Loss: 814856.8661
Epoch 4, Loss: 775606.7666
Epoch 5, Loss: 744873.3567
Epoch 6, Loss: 729293.2649
Epoch 7, Loss: 699784.9513
Epoch 8, Loss: 691439.8746
Epoch 9, Loss: 672116.3438
Epoch 10, Loss: 646672.6937
Epoch 11, Loss: 633821.9253
Epoch 12, Loss: 617288.0042
Epoch 13, Loss: 598315.1306
Epoch 14, Loss: 573739.1544
Epoch 15, Loss: 557521.1606
Epoch 16, Loss: 544370.5834
Epoch 17, Loss: 520475.2927
Epoch 18, Loss: 497779.5022
Epoch 19, Loss: 482544.6814
Epoch 20, Loss: 464988.5220
Epoch 21, Loss: 454491.4145
Epoch 22, Loss: 438711.3097
Epoch 23, Loss: 425661.3795
Epoch 24, Loss: 413898.6615
Epoch 25, Loss: 406642.8397
Epoch 26, Loss: 395747.9575
Epoch 27, Loss: 384835.9787
Epoch 28, Loss: 384322.1367
Epoch 29, Loss: 373317.7943
Epoch 30, Loss: 367012.2460
Epoch 31, Loss: 359958.8377
Epoch 32, Loss: 356391.6339
Epoch 33, Loss: 345026.5919
Epoch 34, Loss: 339160.0112
Epoch 35, Loss: 332751.5156
Epoch 36, Loss: 338202.3310
E

In [23]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 750.2820


In [24]:
torch.save(model.state_dict(), 'neural_network.pth')